In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
from numpy import mean
import warnings
warnings.filterwarnings('ignore')

Importing dataset from Drive

In [ ]:
import pandas as pd
from google.colab import drive
# Mount Google Drive to /content/drive
drive.mount('/content/drive')
# Specify the path to your CSV fila
train = '/content/drive/MyDrive/train1.csv'  # Replace with the actual path to your CSV file
# Read the CSV file into a DataFrame
df1 = pd.read_csv(train)
test = '/content/drive/MyDrive/test1.csv'
df2 = pd.read_csv(test)





Mounted at /content/drive


In [ ]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [ ]:
print("Data Types of Columns:")
print(df1.dtypes)

Data Types of Columns:
full_sq               float64
life_sq               float64
floor                 float64
product_type           object
area_m                float64
                       ...   
mosque_count_5000     float64
leisure_count_5000    float64
sport_count_5000      float64
market_count_5000     float64
price_doc             float64
Length: 271, dtype: object


In [ ]:
#from df2 drop sub_area and row ID
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [ ]:
Xtrain = df1.drop(columns=['price_doc'])
y = df1[['price_doc']]
Xtest = df2


In [ ]:

# Identify categorical columns (type object)
cat_features = Xtrain.select_dtypes(include=['object']).columns
print("\nCategorical Columns:")
print(cat_features)




Categorical Columns:
Index(['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion',
       'incineration_raion', 'oil_chemistry_raion', 'radiation_raion',
       'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion',
       'detention_facility_raion', 'water_1line', 'big_road1_1line',
       'railroad_1line', 'ecology'],
      dtype='object')


In [ ]:
# Identify numerical columns (types int8 or float64)
num_features = Xtrain.select_dtypes(include=['int8', 'float64']).columns
print("\nNumerical Columns:")
print(num_features)



Numerical Columns:
Index(['full_sq', 'life_sq', 'floor', 'area_m', 'raion_popul',
       'green_zone_part', 'indust_part', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       ...
       'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500',
       'cafe_count_5000_price_4000', 'cafe_count_5000_price_high',
       'big_church_count_5000', 'church_count_5000', 'mosque_count_5000',
       'leisure_count_5000', 'sport_count_5000', 'market_count_5000'],
      dtype='object', length=256)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold

Applying log transformation to reduce to reduce skewness of variables

In [ ]:

import numpy as np
# 2. Log Transformation
for col in num_features:
    if col in Xtrain.columns:
        Xtrain[col] = Xtrain[col].apply(lambda x: 0 if x <= 0 else x)  # Handling non-positive values
        Xtrain[col] = Xtrain[col].apply(lambda x: np.log1p(x))  # Log transformation
        Xtest[col] = Xtest[col].apply(lambda x: 0 if x <= 0 else x)  # Handling non-positive values
        Xtest[col] = Xtest[col].apply(lambda x: np.log1p(x))  # Log transformation


In [ ]:

# 1. Label Encoding
label_encoder = LabelEncoder()
for col in cat_features:
    if col in Xtrain.columns:
        Xtrain[col] = label_encoder.fit_transform(Xtrain[col])
        Xtest[col] = label_encoder.fit_transform(Xtest[col])


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer

In [ ]:
# 2. Numerical Feature Transformation
scaler = MinMaxScaler()
Xtrain[num_features] = scaler.fit_transform(Xtrain[num_features])
Xtest[num_features] = scaler.transform(Xtest[num_features])

In [ ]:
# 3. PCA Transformation
pca = PCA(n_components=19)
X_train_pca = pca.fit_transform(Xtrain)
X_test_pca = pca.transform(Xtest)


In [ ]:
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train_pca)
X_test_poly = poly_features.transform(X_test_pca)

In [ ]:
X_train_poly.shape

(181507, 1539)

Training the model with opitimized features

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [ ]:
train_data = lgb.Dataset(X_train_poly, label=y)
test_data = lgb.Dataset(X_test_poly)


In [ ]:
params = {
    'boosting_type': 'gbdt',
      'metric': 'l2',
    'num_leaves': 31,
    'max_depth': -1,
    'learning_rate': 0.05,
    'n_estimators': 100,
      'feature_fraction': 0.9,
    'subsample_for_bin': 200000,
    'objective': 'regression',
    'min_split_gain': 0.0,
    'min_child_weight': 0.001,
    'min_child_samples': 20,
    'subsample': 1.0,
    'subsample_freq': 0,
    'colsample_bytree': 1.0,
    'reg_alpha': 0.0,
    'reg_lambda': 0.0,
    'random_state': 42,
    'importance_type': 'split'
}
# Train the model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 13.248442 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 392445
[LightGBM] [Info] Number of data points in the train set: 181507, number of used features: 1539
[LightGBM] [Info] Start training from score 14751285.292256


In [ ]:

# Make predictions on the test set
y_pred = bst.predict(X_test_poly, num_iteration=bst.best_iteration)

In [ ]:
y_pred.shape

(77789,)

Checking predicted output against output in test file

In [ ]:
import pandas as pd


# Load additional data
additional_data = pd.read_csv('/content/drive/MyDrive/test1.csv')

# Create a DataFrame with 'row ID' and predicted values
predicted_df = additional_data[['row ID']].copy()
predicted_df['price_doc'] = y_pred

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions28.csv', index=False)
